In [ ]:
!pip install medmnist pillow

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.data import IterableDataset, DataLoader
from PIL import Image

from tqdm import tqdm
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
data_flag = 'pathmnist'
# data_flag = 'breastmnist'
download = True

NUM_EPOCHS = [2,5,10]
BATCH_SIZE = 128


info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [ ]:
# Path to the .npz file
npz_file_path = '/root/.medmnist/pathmnist_224.npz'

# Load the .npz file
data = np.load(npz_file_path, mmap_mode='r',allow_pickle=False)

# Check the keys in the .npz file
print("Keys in the .npz file:", data.files)

# Iterate through keys and save each key as a separate .npy file
for key in data.files:
    print(f"Processing and saving key: {key}")

    # Save the data for this key directly to disk
    with open(f"{key}.npy", 'wb') as f:
        np.save(f, data[key])

    print(f"Key '{key}' saved successfully.")

# Close the .npz file after processing
data.close()

Keys in the .npz file: ['train_images', 'train_labels', 'val_images', 'val_labels', 'test_images', 'test_labels']
Processing and saving key: train_images


In [ ]:
class LazyDataset(IterableDataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __iter__(self):
        for idx in range(len(self.dataset)):
            img = self.dataset[idx]  # Lazy loading of the image
            if self.transform:
                img = self.transform(img)  # Apply transformation
            yield img

# Transformation
transform_data = transforms.Compose([
    transforms.ToTensor()  # Converts image to (C, H, W) with values in [0, 1]
])

In [ ]:
# Load .npz file
data = np.load('/root/.medmnist/pathmnist_224.npz')
# Save individual arrays as .npy files for lazy loading
np.save('data.npy', data['data'])
np.save('labels.npy', data['labels'])

# Use memory mapping for individual arrays
data_mmap = np.load('data.npy', mmap_mode='r')
labels_mmap = np.load('labels.npy', mmap_mode='r')

KeyError: 'data is not a file in the archive'

In [ ]:
/root/.medmnist/pathmnist_224.npz

In [ ]:
# Create train and test datasets
train_dataset = DataClass(split='train', transform=None, download=download,size=224, mmap_mode='r')
test_dataset = DataClass(split='test', transform=None, download=download, size=224, mmap_mode='r')

# Wrap datasets with LazyDataset for lazy loading and apply transformations
lazy_train_dataset = LazyDataset(train_dataset, transform=transform_data)
lazy_test_dataset = LazyDataset(test_dataset, transform=transform_data)

# DataLoaders
train_loader = DataLoader(lazy_train_dataset, batch_size=64, pin_memory=True, shuffle=True, num_workers=2)
test_loader = DataLoader(lazy_test_dataset, batch_size=64, pin_memory=True, shuffle=False, num_workers=2)

Using downloaded and verified file: /root/.medmnist/pathmnist_224.npz


In [ ]:
def compute_mean_std(loader):
    mean = 0.0
    std = 0.0
    total_images = 0

    for images, _ in tqdm(loader):
        images = images.view(images.size(0), images.size(1), -1)  # Flatten (C, H, W) to (C, N)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images += images.size(0)

    mean /= total_images
    std /= total_images
    return mean, std

transform_data = transforms.Compose([
    transforms.ToTensor()  # Converts image to (C, H, W) with values in [0, 1]
])

# Load the PathMNIST dataset
train_dataset = DataClass(split='train', transform=transform_data, download=download,size=224, mmap_mode='r')
train_loader = DataLoader(train_dataset, batch_size=64, pin_memory=True,shuffle=False, num_workers=2)

test_dataset = DataClass(split='test', transform=transform_data, download=download, size=224, mmap_mode='r')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

mean_tr, std_tr = compute_mean_std(train_loader)
mean_test, std_test = compute_mean_std(test_loader)

print("Dataset Training Mean:", mean_tr)
print("Dataset Training Std:", std_tr)
print("Dataset Test Mean:", mean_test)
print("Dataset Test Std:", std_test)

Using downloaded and verified file: /root/.medmnist/pathmnist_224.npz


In [ ]:
# Function to compute PCA noise
def add_pca_noise(img, alpha_std=0.1):
    """
    Add PCA noise to an image.
    Args:
        img: A PIL Image or Tensor (C x H x W).
        alpha_std: The standard deviation of the noise to add.
    Returns:
        A Tensor with PCA noise added.
    """
    if isinstance(img, Image.Image):  # If the input is a PIL Image
        img = transforms.ToTensor()(img)  # Convert to Tensor (C, H, W)

    img_np = img.permute(1, 2, 0).numpy()  # Convert to NumPy (H, W, C)
    img_flat = img_np.reshape(-1, 3)  # Reshape to (N, 3) where N = H*W

    # Compute mean and covariance
    mean = np.mean(img_flat, axis=0)
    centered = img_flat - mean
    cov = np.cov(centered, rowvar=False)

    # Eigen decomposition
    eigvals, eigvecs = np.linalg.eigh(cov)

    # Add noise along the principal components
    noise = np.random.normal(0, alpha_std, size=3) * eigvals
    noise = eigvecs @ noise  # Project noise into RGB space

    img_flat += noise  # Add noise to each pixel
    img_flat = np.clip(img_flat, 0, 1)  # Clip values to valid range
    img_noisy = img_flat.reshape(img_np.shape)  # Reshape back to original

    return torch.tensor(img_noisy).permute(2, 0, 1)  # Convert back to Tensor (C, H, W)

# preprocessing
train_data_transform = transforms.Compose([
    # Random crop and resize
    transforms.RandomResizedCrop(size=224, scale=(0.08, 1.0), ratio=(3/4, 4/3)),

    # Random horizontal flip
    transforms.RandomHorizontalFlip(p=0.5),

    # Color jitter
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),

    # PCA noise
    transforms.Lambda(lambda img: add_pca_noise(img, alpha_std=0.1)),

    # Normalize
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean_tr[0].item(),mean_tr[1].item(),mean_tr[2].item()], std=[std_tr[0].item(), std_tr[1].item(), std_tr[2].item()])
])

# Define test dataset transformations using calculated mean and std
test_data_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean_test[0].item(), mean_test[1].item(), mean_test[2].item()],
                         std=[std_test[0].item(), std_test[1].item(), std_test[2].item()])
])

# load the data
train_dataset = DataClass(split='train', transform=train_data_transform, download=download, size=224, mmap_mode='r')
test_dataset = DataClass(split='test', transform=test_data_transform, download=download, size=224, mmap_mode='r')

#pil_dataset = DataClass(split='train', download=download)

# Define the sizes for train and validation splits
#train_size = len(train_dataset)  # 80% of the data
#val_size = len(train_dataset) - train_size  # Remaining 20%

# Split the train_dataset into training and validation sets
#train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders for training and validation sets
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#val_loader = data.DataLoader(dataset=val_subset, batch_size=2*BATCH_SIZE, shuffle=True)

# Test DataLoader remains unchanged
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz


In [ ]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset PathMNIST of size 28 (pathmnist)
    Number of datapoints: 89996
    Root location: /root/.medmnist
    Split: train
    Task: multi-class
    Number of channels: 3
    Meaning of labels: {'0': 'adipose', '1': 'background', '2': 'debris', '3': 'lymphocytes', '4': 'mucus', '5': 'smooth muscle', '6': 'normal colon mucosa', '7': 'cancer-associated stroma', '8': 'colorectal adenocarcinoma epithelium'}
    Number of samples: {'train': 89996, 'val': 10004, 'test': 7180}
    Description: The PathMNIST is based on a prior study for predicting survival from colorectal cancer histology slides, providing a dataset (NCT-CRC-HE-100K) of 100,000 non-overlapping image patches from hematoxylin & eosin stained histological images, and a test dataset (CRC-VAL-HE-7K) of 7,180 image patches from a different clinical center. The dataset is comprised of 9 types of tissues, resulting in a multi-class classification task. We resize the source images of 3×224×224 into 3×28×28, and split NCT-CRC-HE-100K

In [ ]:
list(Xtr[0])[0].shape #1st image of train_dataset

torch.Size([3, 32, 32])

In [ ]:
class ResidualBlock(nn.Module):
    """Basic Residual Block."""
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = F.relu(out)

        return out

In [ ]:
import torch.nn.init as init
class ResNet(nn.Module):
    """ResNet Architecture."""
    def __init__(self, block, layers, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64

        # Input Stem
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual Layers
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # Final Output Layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [ ]:
def initialize_weights(module):
    """
    Initialize weights of the module according to the Xavier algorithm.
    """
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        # Xavier initialization
        fan_in = module.weight.size(1)
        fan_out = module.weight.size(0)
        a = math.sqrt(6 / (fan_in + fan_out))
        nn.init.uniform_(module.weight, -a, a)
        if module.bias is not None:
            nn.init.constant_(module.bias, 0)
    elif isinstance(module, nn.BatchNorm2d):
        # BatchNorm initialization
        nn.init.constant_(module.weight, 1)  # gamma = 1
        nn.init.constant_(module.bias, 0)    # beta = 0

In [ ]:
model = ResNet(ResidualBlock, [3,4,6,3] ,num_classes=n_classes).to(device)
model.apply(initialize_weights) #apply the initializer to the entire model
# define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

#Define initial learning rate and batch size
initial_lr = 0.1
base_batch_size = 256
current_batch_size = BATCH_SIZE

scaled_lr = initial_lr * (current_batch_size / base_batch_size)
# Define optimizer with the scaled learning rate
optimizer = optim.SGD(model.parameters(), lr=scaled_lr, momentum=0.9, weight_decay=1e-4)

### Using Simple CNN

In [ ]:
# define a simple CNN model

class Net(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            #nn.Dropout(p=0.3) #Change
            )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Sequential(
            nn.Linear(64 * 5 * 5, 128),
            nn.ReLU(),
            #nn.Dropout(p=0.5), #Change
            nn.Linear(128, 128),
            nn.ReLU(),
            #nn.Dropout(p=0.5),  #Change
            nn.Linear(128, num_classes)
            )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = Net(in_channels=n_channels, num_classes=n_classes).to(device)
#model.apply(initialize_weights)

# define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9,weight_decay=1e-4)

In [ ]:
model = ResNet(ResidualBlock, [3,4,6,3] ,in_channels=n_channels, num_classes=n_classes).to(device)
# define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay = 1e-4)

In [ ]:
for epoch in range(NUM_EPOCHS[2]):
    train_correct = 0
    train_total = 0
    total_loss = 0

    # Training Phase
    model.train()
    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(inputs)

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            loss = criterion(outputs, targets)
        else:
            targets = targets.squeeze().long()
            loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    train_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Training Loss: {train_loss:.4f}')

In [ ]:
# Validation Phase
    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                loss = criterion(outputs, targets)
            else:
                targets = targets.squeeze().long()
                loss = criterion(outputs, targets)

            total_val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    val_loss = total_val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

In [ ]:
model.eval()
correct = 0
total = 0
val_loss = 0
with torch.no_grad():
    for inputs, targets in tqdm(test_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            loss_test = criterion(outputs, targets)
        else:
            targets = targets.squeeze().long()
            loss_test = criterion(outputs, targets)
        val_loss += loss_test.item()
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
val_accuracy = 100 * correct / total
print(f'test Loss: {val_loss / len(test_loader):.4f}, Test Accuracy: {val_accuracy:.2f}%')


100%|██████████| 57/57 [00:02<00:00, 26.21it/s]

test Loss: 0.6917, Test Accuracy: 80.86%
